In [14]:
import io
import json
import os
import sys
import random
import csv
import subprocess
import numpy as np
import pandas as pd
import tifffile as tiff
from PIL import Image
from math import sqrt, exp, log
from matplotlib import cm
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import shutil
from flir_image_extractor import FlirImageExtractor
from scipy.ndimage import zoom

In [15]:
os.chdir(sys.path[0])

# /Volumes/SANDISK64GB/GitHub/Tesi_magistrale_Bugin/data_source
data_base_folder = os.path.join(
    '..', 'Tesi_magistrale_bugin', 'data_source', 'archivio_foto_tesi')

In [5]:
fie = FlirImageExtractor(exiftool_path='exiftool')

for current_folder, _, _ in os.walk(data_base_folder):
    jpg_file_list = sorted([os.path.join(current_folder, name) for name in os.listdir(current_folder) if
                            (not name.startswith('.')) and
                            name.endswith('.jpg') and
                            os.path.isfile(os.path.join(current_folder, name))])
    for jpg_file in jpg_file_list:
        processing_ok = True
        try:
            fie.process_image(jpg_file)
        except Exception as e:
            print(f"An error occurred calling fie.process_image(): {e}")
            processing_ok = False
        try:
            meta = fie.get_metadata()
        except Exception as e:
            print(f"An error occurred calling fie.get_metadata(): {e}")
            processing_ok = False
        try:
            thermal_channel = fie.get_thermal_np()
        except Exception as e:
            print(f"An error occurred calling fie.get_thermal_np(): {e}")
            processing_ok = False
        try:
            rgb_channels = fie.get_rgb_np()
        except Exception as e:
            print(f"An error occurred calling fie.get_rgb_np(): {e}")
            processing_ok = False
    
        if processing_ok:
            dest_npy_file = os.path.join(
                os.path.join('archivio_foto_tesi',
                             'pure_rgb_matrices_(480,640,3)'),
                f'{Path(jpg_file).stem}.npy')
            np.save(dest_npy_file, rgb_channels)
            dest_npy_file = os.path.join(
                os.path.join('archivio_foto_tesi',
                            'pure_thermal_matrices_(60,80)'),
                f'{Path(jpg_file).stem}.npy')
            np.save(dest_npy_file, thermal_channel)
            dest_json = os.path.join(
                os.path.join('archivio_foto_tesi',
                             'metadata'),
                f'{Path(jpg_file).stem}.json')
            with open(dest_json, 'w') as json_file:
                json.dump(meta, json_file, indent=4)


In [36]:
def resize(thermal_channel, dest_shape, metadata):
    Real2IR = metadata['Real2IR']
    # Define grid for interpolation
    # order=1 corresponds to linear interpolation
    thermal_channel_new = zoom(thermal_channel, zoom=(
        Real2IR, Real2IR), order=1)
    print(f'thermal_channel_new.shape == {thermal_channel_new.shape}')
    print(f'thermal_channel_new.dtype == {thermal_channel_new.dtype}')
    H_scaled, W_scaled = thermal_channel_new.shape
    H_new, W_new, _ = dest_shape
    thermal_channel_new_with_nan = np.full((H_new, W_new), np.nan)
    OffsetX = metadata['OffsetX']
    OffsetY = metadata['OffsetY']
    print(f'OffsetX == {OffsetX}')
    print(f'OffsetY == {OffsetY}')
    print(
        f'thermal_channel_new_with_nan.shape == {thermal_channel_new_with_nan.shape}')
    print(
        f'thermal_channel_new_with_nan.dtype == {thermal_channel_new_with_nan.dtype}')
    # Assign values from thermal_channel_new into thermal_channel_new_with_nan starting at (OffsetX, OffsetY)
    thermal_channel_new_with_nan[OffsetX:OffsetX+thermal_channel_new.shape[0], OffsetY:OffsetY+thermal_channel_new.shape[1]] = thermal_channel_new

    return thermal_channel_new_with_nan

In [37]:
fie = FlirImageExtractor(exiftool_path='exiftool')

stop = False

for current_folder, _, _ in os.walk(data_base_folder):
    jpg_file_list = sorted([os.path.join(current_folder, name) for name in os.listdir(current_folder) if
                            (not name.startswith('.')) and
                            name.endswith('.jpg') and
                            os.path.isfile(os.path.join(current_folder, name))])
    if stop:
        break# just when debugging

    for jpg_file in jpg_file_list:
        metadata = None
        rgb_channels_norm = None
        thermal_channel_norm = None

        processing_ok = True
        try:
            fie.process_image(jpg_file)
        except Exception as e:
            print(f"An error occurred calling fie.process_image(): {e}")
            processing_ok = False
        try:
            metadata = fie.get_metadata()
        except Exception as e:
            print(f"An error occurred calling fie.get_metadata(): {e}")
            processing_ok = False
        try:
            rgb_channels = fie.get_rgb_np()
            rgb_channels_norm = (rgb_channels.astype(np.float32))/255.0
        except Exception as e:
            print(f"An error occurred calling fie.get_rgb_np(): {e}")
            processing_ok = False
        try:
            thermal_channel = fie.get_thermal_np()
        except Exception as e:
            print(f"An error occurred calling fie.get_thermal_np(): {e}")
            processing_ok = False

        if processing_ok:
            min_temp = np.min(thermal_channel)
            max_temp = np.max(thermal_channel)
            delta_temp = abs(max_temp-min_temp)
            thermal_channel_norm = (thermal_channel.astype(
                np.float32) - min_temp)/delta_temp
            # thermal_channel has size (60, 80)
            # rgb_channels_norm has shape (480, 640, 3)
            # thermal_channel_norm is resized to (H, W), with 60 < H < 480 and 80 < W < 640
            # finally thermal_channel_norm is put inside an outer frame of np.nan values
            # to resize it to (480, 640)
            thermal_channel_norm_resized = resize(
                thermal_channel, rgb_channels_norm.shape, metadata)
            cmap = plt.colormaps.get_cmap('rainbow')  # or 'nipy_spectral'
            thermal_channel_norm_as_rgb = cmap(
                thermal_channel_norm_resized)[:, :, :3] #this is normalized
            plt.title(f'{Path(jpg_file).stem}')
            plt.imshow(thermal_channel_norm_as_rgb)
            plt.show()
            alpha = 0.2
            thermally_enhanced_rgb = (1-alpha)*rgb_channels_norm + \
                alpha * thermal_channel_norm_as_rgb
            plt.title(f'{Path(jpg_file).stem}')
            plt.imshow(thermally_enhanced_rgb)
            plt.show()
            
            """
            image = Image.fromarray((thermally_enhanced_rgb*255).astype(
                np.uint8))  # Ensure the array is of type uint8
            dest_jpg_file = os.path.join(
                os.path.join('archivio_foto_tesi',
                             'temperature_enhanced_images_(...,...,3)'),
                f'{Path(jpg_file).stem}.jpg')
            image.save(dest_jpg_file)
            """
            stop = True
            
            

thermal_channel_new.shape == (84, 112)
thermal_channel_new.dtype == float64
OffsetX == 0
OffsetY == -12
thermal_channel_new_with_nan.shape == (480, 640)
thermal_channel_new_with_nan.dtype == float64


ValueError: could not broadcast input array from shape (84,112) into shape (84,0)